In [1]:
import sys 
sys.path.append(r"C:\Users\a-sochat\development\Qcodes_Sohail_Clone\Qcodes")

import numpy as np
import time 

import qcodes as qc
from qcodes.instrument.parameter import ManualParameter

from qcodes.sweep import sweep, nest, chain, szip, getter, setter
from qcodes.sweep.sweep import BaseSweepObject, ParametersTable, wrap_objects, time_trace

from qcodes.sweep.sweep import (
    Nest, Zip, Chain, ParameterSweep, ParameterWrapper, FunctionSweep, FunctionWrapper
)

C:\Users\a-sochat\AppData\Local\Continuum\Anaconda3\envs\qcodes\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
class Printer:
    def __init__(self, sweep_object): 
        self._ind, self._dep = sweep_object.parameter_table.flatten()
        self._symbols_list = sweep_object.parameter_table.symbols_list()
        self._inferred_symbols_list = sweep_object.parameter_table.inferred_symbols_list()
        
    def __enter__(self): 
        header_ind = "\t".join(["{} [{}]".format(*i) for i in self._ind.items()])
        header_dep = "\t".join(["{} [{}]".format(*i) for i in self._dep.items()])
        sep = " | "
        
        print(self._inferred_symbols_list)
        print((header_ind + sep + header_dep).strip(sep))
        
        return self
    
    def __exit__(self, type, value, traceback): 
        pass

    def __call__(self, result):
        print(" " + "\t ".join([str(result[ip]) for ip in self._symbols_list]))

Lets define some parameters and functions to sweep over

In [6]:
x = ManualParameter("x", unit="V")
y = ManualParameter("y", unit="V")

m = ManualParameter("m", unit="A")
m.get = lambda: x() ** 2

n = ManualParameter("n", unit="A")
n.get = lambda: x() - y() ** 2 + 16

Convinience functions and other [syntactic sugar](https://en.wikipedia.org/wiki/Syntactic_sugar) has been developed to simplify writing complext loops. The following sweep objects as equivalent: 

In [7]:
# Can someone understand what the hell this means? 
sweep_object_1 = Nest([
    ParameterSweep(x, lambda: [0, 1, 2]), 
    Chain([
        ParameterWrapper(m), 
        Nest([
            ParameterSweep(y, lambda: [0, 2, 4]), 
            ParameterWrapper(n)
        ])
    ])
])
# Lets unloop to see what it does 
with Printer(sweep_object_1) as printer:
    for i in sweep_object_1:   
        printer(i)


x [V]	y [V] | m [A]	n [A]
 0	 None	 0	 None
 0	 0	 None	 16
 0	 2	 None	 12
 0	 4	 None	 0
 1	 None	 1	 None
 1	 0	 None	 17
 1	 2	 None	 13
 1	 4	 None	 1
 2	 None	 4	 None
 2	 0	 None	 18
 2	 2	 None	 14
 2	 4	 None	 2


In [8]:
sweep_object_1._parameter_table


x [V]|m [A]
x [V],y [V]|n [A]

In [10]:
# Lets write the same thing in a slightly more convinient way 
sweep_object_2 = nest(
    sweep(x, [0, 1, 2]), 
    chain(
        m, 
        nest(
            sweep(y,[0, 2, 4]), 
            n
        )
    )
)

# Lets unloop to see what it does 
with Printer(sweep_object_2) as printer:
    for i in sweep_object_2:   
        printer(i)


x [V]	y [V] | m [A]	n [A]
 0	 None	 0	 None
 0	 0	 None	 16
 0	 2	 None	 12
 0	 4	 None	 0
 1	 None	 1	 None
 1	 0	 None	 17
 1	 2	 None	 13
 1	 4	 None	 1
 2	 None	 4	 None
 2	 0	 None	 18
 2	 2	 None	 14
 2	 4	 None	 2


### what we lean: 
1) The function sweep (with small "s") either returns a Parameter sweep or a Function sweep class depending on the argument types

2) The range argument for the *classes* Parameter sweep and Function sweep are lambda functions, while for the sweep *function* this can be a simple list or numpy array (or infact any other iterable such as a generator)

3) The arguments to the Nest and Chain classes are lists, while for the nest and chain functions are function arguments

4) While the Parameter sweep and Function sweep classes expect arguments of qcodes parameter type and callables to be wrapped by ParameterWrapper and FunctionWrapper respectively, the sweep functions wraps these automatically

In [11]:
# Finally, we can also just write...
sweep_object = sweep(x, [0, 1, 2])(
    m,
    sweep(y, [0, 2, 4])(
        n  
    )
)

with Printer(sweep_object) as printer:
    for i in sweep_object:   
        printer(i)


x [V]	y [V] | m [A]	n [A]
 0	 None	 0	 None
 0	 0	 None	 16
 0	 2	 None	 12
 0	 4	 None	 0
 1	 None	 1	 None
 1	 0	 None	 17
 1	 2	 None	 13
 1	 4	 None	 1
 2	 None	 4	 None
 2	 0	 None	 18
 2	 2	 None	 14
 2	 4	 None	 2


### what we lean: 
1) A sweep object is callable. We have the following rule: `sweep_object(m)` is the same as `nest(sweep_object, m)`. This saves us from writing "nest".

2) Furthermore, `sweep_object(m, n)` is the same as `nest(sweep_object, chain(m, n))`. Implicit chaining reduces the bracket level and saves us from writing "chain". 